# Quantum Chemistry Demo

In [18]:
#Import modules and define SMILES
from ase.io import read, write
import os
import subprocess
import nglview as nv
import MDAnalysis as mda
import nglview as nv
from nglview.datafiles import PDB, XTC
from rdkit import Chem
from rdkit.Chem import AllChem
import py3Dmol
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [14]:
# Define your molecules (names and SMILES)
molecules = {
    "water": "O",
    "ammonia": "N",
    "hydrogen_fluoride": "[H]F",
    "carbon_monoxide": "[C-]#[O+]",  # better charge-separated resonance structure
    "methane": "C"
}

In [28]:
# Loop through each molecule and create input files
for name, smiles in molecules.items():
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)  # Add hydrogens
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)

    # Extract XYZ coordinates
    conf = mol.GetConformer()
    xyz_block = ""
    for atom in mol.GetAtoms():
        pos = conf.GetAtomPosition(atom.GetIdx())
        xyz_block += f"{atom.GetSymbol()} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}\n"

    # Format ORCA input
    orca_input = f"""! HF def2-SVP TightSCF

* xyz 0 1
{xyz_block}*
"""

    # Write to file
    filename = f"{name}.inp"
    with open(filename, "w") as f:
        f.write(orca_input)

    print(f"ORCA input file '{filename}' written.")

ORCA input file 'water.inp' written.
ORCA input file 'ammonia.inp' written.
ORCA input file 'hydrogen_fluoride.inp' written.
ORCA input file 'carbon_monoxide.inp' written.
ORCA input file 'methane.inp' written.


## ORCA Input Line Explanation

### `! HF def2-SVP TightSCF`

This command line specifies the type of quantum chemical calculation to be performed by ORCA. Each keyword sets a method or option:

- **`HF`**  
Refers to the Hartree–Fock method, a foundational ab initio quantum chemistry approach. It solves the electronic Schrödinger equation using a single Slater determinant, approximating electron–electron interactions in a mean-field sense. HF does not include electron correlation beyond exchange, making it less accurate than DFT or post-HF methods, but useful for basic property predictions, benchmarking, or method comparisons.

- **`def2-SVP`**  
A split-valence basis set from the def2 family developed by Ahlrichs and coworkers. The “SVP” stands for split valence with polarization, which means:
	•	Valence orbitals are represented by multiple basis functions (improving flexibility)
	•	Polarization functions are added to allow for angular distortion of electron clouds
This basis set offers a good compromise between computational efficiency and reasonable accuracy for small to medium-sized molecules.

- **`TightSCF`**  
Specifies tight convergence criteria for the Self-Consistent Field (SCF) procedure. During SCF, ORCA iteratively solves for the electronic wavefunction until it converges. TightSCF reduces the threshold for convergence, ensuring higher precision in energy and property calculations, which is particularly important when computing derivatives like dipole moments or in preparation for subsequent steps (e.g., frequency analysis).

##  Molecular geometry section

<pre>
* xyz 0 1

This marks the beginning of the molecular geometry section.

• <b>xyz:</b> Geometry format used — atomic symbols and Cartesian coordinates (x, y, z).
• <b>0:</b> Total charge of the system (here, neutral).
• <b>1:<b> Spin multiplicity (here, singlet, meaning all electrons are paired).

*

This asterisk marks the end of the coordinate input section.
</pre>

In [29]:
# List of molecule names matching your .inp files (without extensions)
molecule_names = ["water", "ammonia", "hydrogen_fluoride", "carbon_monoxide", "methane"]

# Ensure a main output directory exists
os.makedirs("outputs", exist_ok=True)

for name in molecule_names:
    inp_file = f"{name}.inp"
    out_file = f"{name}.out"
    molecule_output_dir = f"outputs/{name}_files"

    # Run ORCA
    print(f"Running ORCA for {name}...")
    subprocess.run(["orca", inp_file], stdout=open(out_file, "w"))

    # Create directory for molecule-specific files
    os.makedirs(molecule_output_dir, exist_ok=True)

    # Move related files into the folder
    for ext in ["inp", "out", "gbw", "prop", "xyz","property.txt","densitiesinfo", "densities", "bibtex"]:
        file_path = f"{name}.{ext}"
        if os.path.exists(file_path):
            os.rename(file_path, os.path.join(molecule_output_dir, f"{name}.{ext}"))

    print(f"Finished {name}, files stored in '{molecule_output_dir}'.")

print("All ORCA jobs completed.")

Running ORCA for water...
Finished water, files stored in 'outputs/water_files'.
Running ORCA for ammonia...
Finished ammonia, files stored in 'outputs/ammonia_files'.
Running ORCA for hydrogen_fluoride...
Finished hydrogen_fluoride, files stored in 'outputs/hydrogen_fluoride_files'.
Running ORCA for carbon_monoxide...
Finished carbon_monoxide, files stored in 'outputs/carbon_monoxide_files'.
Running ORCA for methane...
Finished methane, files stored in 'outputs/methane_files'.
All ORCA jobs completed.


In [30]:
# Extract Dipole moments from .out files
base_dir = "outputs"
dipole_results = {}

# Loop through each subdirectory (e.g., water_files, ammonia_files)
for subfolder in os.listdir(base_dir):
    subfolder_path = os.path.join(base_dir, subfolder)
    if not os.path.isdir(subfolder_path):
        continue

    # Look for the .out file (same name as subfolder but without "_files")
    molecule_name = subfolder.replace("_files", "")
    out_file = os.path.join(subfolder_path, f"{molecule_name}.out")

    if not os.path.exists(out_file):
        dipole_results[molecule_name] = "Output file not found"
        continue

    dipole_debye = None
    with open(out_file, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if "Magnitude (Debye)" in line:
                try:
                    dipole_debye = float(line.split(":")[1].strip())
                except:
                    dipole_debye = "Parse error"
                break

    dipole_results[molecule_name] = dipole_debye if dipole_debye is not None else "Not found"

# Print summary
print("Dipole Moments (Debye):")
for mol, value in dipole_results.items():
    print(f"{mol:<20}: {value}")

Dipole Moments (Debye):
water               : 2.166625101
hydrogen_fluoride   : 2.128157275
methane             : 1.1488e-05
carbon_monoxide     : 0.226747896
ammonia             : 1.797098994
